### UPDATED 10-29-2020

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/lopezbec/Traffic_Accident_Detection/blob/master/Yolov3_DeepSort/GPU/Implementation_Object_tracker.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

#### To run this code you just need to follow the instruction and run each of the code cell by clicking on the “play” button 

### Housekeeping Notes:

Before you start working with this notebook, you need decide if you would like to test the code with the video on the GitHub repository or with a new Video.

If you would like to run a new video (i.e., your own video) , you would need to clone the repository to your own Google Drive. Hence, first you would need to  "mount" your Google Drive in the runtime's Virtual Machine. This will allow you to read and save files from your own Google Drive "Traffic_Accident_Detection" folder. 

To do this, you need to provide the path of your Google Drive for the folder of this assignment and click on the option of “Use_Google_Drive”. Once you provide this information, run this code cell below and follow the instructions. If you don’t click Use_Google_Drive, there is not need to submit a “Working_Directory”


In [3]:

Use_Google_Drive= False #@param {type:"boolean"}

if Use_Google_Drive:
    import os
    from google.colab import drive 

    drive.mount('/content/gdrive',force_remount=True)

    Working_Directory = 'My Drive' #@param {type:"string"}
    wd="/content/gdrive/"+Working_Directory
    os.chdir(wd)

    !git clone https://github.com/lopezbec/Traffic_Accident_Detection

    
    %cd Traffic_Accident_Detection/Yolov3_DeepSort/GPU 
    dirpath = os.getcwd()
    print("current directory is : " + dirpath)

    
else:
    !git clone https://github.com/lopezbec/Traffic_Accident_Detection
    %cd Traffic_Accident_Detection/Yolov3_DeepSort/GPU 




Cloning into 'Traffic_Accident_Detection'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 122 (delta 26), reused 89 (delta 13), pack-reused 0
Receiving objects: 100% (122/122), 38.91 MiB | 43.22 MiB/s, done.
Resolving deltas: 100% (26/26), done.
/content/Traffic_Accident_Detection/Yolov3_DeepSort/GPU


Before we begin, we need to install all the requirements from the `requirements-gpu.txt` file 

In [4]:
#installing requirements
!pip install -r requirements-gpu.txt
from IPython.display import clear_output
#clear_output()
print("All Requirements were installed!")

     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 28.7MB 100kB/s 
     |████████████████████████████████| 450kB 54.6MB/s 
     |████████████████████████████████| 3.9MB 51.6MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=07712853b6b390031eec48c83326e67298141cb7ea81b5d8335192bdcb6cec94
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll hav

All Requirements were installed!


We need to also download the weight files, that are not on the repository (this will take several minutes, ~30min)

In [5]:
#Downloading the weigts
!wget https://pjreddie.com/media/files/yolov3.weights -O weights/yolov3.weights

--2020-11-10 02:11:01--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘weights/yolov3.weights’

weights/yolov3.weig 100%[===================>] 236.52M  42.7KB/s    in 72m 53s 

2020-11-10 03:23:56 (55.4 KB/s) - ‘weights/yolov3.weights’ saved [248007048/248007048]



**Loading the weight files** 

In [6]:
# yolov3
!python load_weights.py
#clear_output()
print("Weights were loaded!")

2020-11-10 03:23:58.038965: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2020-11-10 03:23:58.039122: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2020-11-10 03:23:58.039142: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2020-11-10 03:23:59.732549: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-10 03:23:59.792325: I tensorflow/stream_executor/cuda/cuda_g

#**Runing the tracker**
You can run the tracker setting the video input and the output path for the resulting video.

Note: for this case the resulting video is allways going to be a .avi file.

Everytime the tracker is excecuted it is going to create/update a ".json" file containing:


*   Frame: the number of frame where the detection is made from.
*   Track_id: an identification given to every object detected in the video.
*   Class: the class of object detected.
*   Center: this is going to be a set of coordinates x and y pointing to the center of the object detected in every case.
*   Detection_box: this is a set of tow pairs of coordinates x and y, pointing to the starting and ending point for the rectangle drawned around every detection, this is going to be the next way (start_x, start_y,end_x,endy).


In order to run your own video, you need to upload the video to the `Traffic_Accident_Detection/Yolov3_DeepSort/GPU/data/video` folder of your Google Drive and change the name in this code OR change the name of the video to `test.mp4`

In [7]:
# yolov3 on video
!python object_tracker.py --video ./data/video/test.mp4 --output ./data/video/results.mp4
#clear_output()
print('''
        Video object tracking is done, go to /data/video
        to see the resulting video...
''')

2020-11-10 03:24:18.021435: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2020-11-10 03:24:18.021561: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2020-11-10 03:24:18.021586: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2020-11-10 03:24:19.136942: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-10 03:24:19.172386: I tensorflow/stream_executor/cuda/cuda_g

Now you can see the output video. Since Google Collab does not easily allows to play video you can either download the video to your computer, see it from your Google Drive folder (if you enalbed that option) or see each frame of the video independently.

In [22]:
%cd data/video

Download_video_and_json= True #@param {type:"boolean"}

if Download_video_and_json:
    from google.colab import files
    files.download('results.mp4') 
    %cd ..
    %cd ..
    files.download('info.json') 
else:
    from google.colab.patches import cv2_imshow
    import cv2

    # Download sample video


    cap = cv2.VideoCapture('results.mp4')
    while cap.isOpened():
        ret, image = cap.read()

        if not ret:
            break

        cv2_imshow(image) # Note cv2_imshow, not cv2.imshow

        cv2.waitKey(1) & 0xff

    cv2.destroyAllWindows()
    cap.release()

/content/Traffic_Accident_Detection/Yolov3_DeepSort/GPU/data/video


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/content/Traffic_Accident_Detection/Yolov3_DeepSort/GPU/data
/content/Traffic_Accident_Detection/Yolov3_DeepSort/GPU


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>